# U.K. Measles and MMR Vaccine Tracking Dashboard

This dashboard analyzes public health data from the United Kingdom Health Security Agency (UKHSA), specifically exploring measles cases and MMR vaccination coverage percentages in England over several years. 

--------

In [10]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import json

In [11]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [12]:
# Loads JSON files and store the raw data in a variable as a dictionary.

with open("measles_cases.json", "rt") as INFILE:
    cases_jsondata={"measles_cases": json.load(INFILE)}
with open("mmr1_coverage.json", "rt") as INFILE:
    mmr1_jsondata={"mmr1_coverage": json.load(INFILE)} 
with open("mmr2_coverage.json", "rt") as INFILE:
    mmr2_jsondata={"mmr2_coverage": json.load(INFILE)}

In [13]:
# Below function wrangles the raw data from the JSON file cases_jsondata into a DataFrame that will be used for plotting 

def wrangle_data_measles_age(measles_cases):
    """ Parameters: measles_cases - data from json file. Returns a dataframe.
    Includes the code that wrangles the data, creates the dataframe and fills it in. """    
    # Measles cases by week onset metric value, date, and age
    data={}
    for entry in measles_cases:
        date=entry['date']
        age=entry['age'] # Collecting age as the 'metric'
        value=entry['metric_value'] # Still collecting the actual metric (measles cases)
        if age=='all': # Skips 'all' entry for age since not valuable metric to ages data
            continue
        if date not in data:
            data[date]={}
        data[date][age]=value
    # Parse and sort dates
    dates=sorted(data.keys())
    startdate=pd.to_datetime(dates[0])
    enddate=pd.to_datetime(dates[-1])
    # Parse and sort age
    ages=[]
    for entry in data.values():
        for age in entry.keys():
            if age not in ages:
                ages.append(age)
    ages.sort()
    # Create the DataFrame
    index=pd.date_range(startdate, enddate, freq='W-MON') # W-MON "weekly on Mondays" (when data is updated)
    df=pd.DataFrame(index=index, columns=ages)
    # Insert the data into the DataFrame
    for date, entry in data.items(): # Each entry is a dictionary with some ages and values
        pd_date=pd.to_datetime(date) 
        for column in entry.keys(): # The ages
            df.loc[pd_date, column]=entry[column] #.loc to access a specific location in the DataFrame
    df.fillna(0.0, inplace=True) # Filling in any remaining "holes" due to missing dates and ages
    df.infer_objects(copy=False) # Adjust types of columns filled in                
    return df

adf=wrangle_data_measles_age(cases_jsondata["measles_cases"]) # adf is the DataFrame for plotting

/tmp/ipykernel_3361/1256550371.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0.0, inplace=True) # Filling in any remaining "holes" due to missing dates and ages


In [14]:
# Below function wrangles the raw data from the JSON files mmr1_jsondata and mmr2_jsondata into a DataFrame that will be used for plotting 

def wrangle_data_mmr(mmr1_coverage, mmr2_coverage):
    """ Parameters: mmr1_coverage, mmr2_coverage - data from json file. Returns a dataframe.
    Includes the code that wrangles the data, creates the dataframe and fills it in. """
    # MMR1 & MMR2 coverage percentages metric value, metric, and date
    data={}
    for entry in mmr1_coverage:  
        date=entry['date']
        metric=entry['metric']
        value=entry['metric_value']
        if date not in data:
            data[date]={}
        data[date][metric]=value
    for entry in mmr2_coverage: 
        date=entry['date']
        metric=entry['metric']
        value=entry['metric_value']
        if date not in data:
            data[date]={}
        data[date][metric]=value
    # Parse and sort dates
    dates=sorted(data.keys())
    startdate=pd.to_datetime(dates[0])
    enddate=pd.to_datetime(dates[-1])
    #Create the DataFrame
    index=pd.date_range(startdate, enddate, freq='YE-MAR') # Yearly repeating in March 
    df=pd.DataFrame(index=index, columns=['MMR1', 'MMR2'])
    # Translate the columns to the metrics and inserting the data into DataFrame
    metrics={'MMR1': 'MMR1_coverage_coverageByYear', 'MMR2': 'MMR2_coverage_coverageByYear'}
    for date, entry in data.items(): 
        pd_date=pd.to_datetime(date) 
        for column in ['MMR1', 'MMR2']: 
            metric_name=metrics[column]
            value=entry.get(metric_name, 0.0)
            df.loc[date, column]=value
    df.fillna(0.0, inplace=True)
    return df

df=wrangle_data_mmr(mmr1_jsondata["mmr1_coverage"], mmr2_jsondata["mmr2_coverage"]) # df is the DataFrame for plotting

## Download the Current Data

The current trends for data updates by UKHSA are as follows: 
- Data for measles cases are updated once a week on Mondays.
- Data for MMR vaccination coverage percentages are updated once a year in March.

Note these update trends are not guaranteed to continue as indicated and the user may see deviations from these trends. 

Below is a refresh button to *optionally* refresh the data to the current collected data set. After the data has successfully refreshed, the button will turn green and indicate to the user that the data has been updated. The user will see the updated data in the graphs in the Graphs and Analysis section. 

In the event that the server cannot be reached or the data is unavailable, the refresh button will turn red and indicate such information to the user by displaying "Unavailable". Note that this is no cause for concern as the graphs will be loaded from stored data. If the user wishes, they can try to refresh again later. 

In [15]:
# Below functions download the current data to give the user an option to refresh the dataset in conjunction with the refresh button

# Uses APIwrapper which is imported from a separate file
from APIwrapper import APIwrapper

def access_api_measles(): # Access API data from UKHSA for measles cases
    """ Accesses the UKHSA API for measles cases. Returns data as a like-for-like 
    replacement for the "canned" data loaded from the JSON file. """
    api=APIwrapper(theme="infectious_disease", # Structure that contains the main parameters of the query  
                     sub_theme="vaccine_preventable",
                     topic="Measles",
                     geography_type="Nation",
                     geography="England", 
                     metric="measles_cases_casesByOnsetWeek")
    measles_cases=api.get_all_pages()
    return measles_cases # Return data read from the API

def access_api_mmr1(): # Access API data from UKHSA for MMR1 percentages
    """ Accesses the UKHSA API for MMR1. Returns data as a like-for-like replacement 
    for the "canned" data loaded from the JSON file. """
    api=APIwrapper(theme="immunisation", # Structure that contains the main parameters of the query  
                     sub_theme="childhood-vaccines",
                     topic="MMR1",
                     geography_type="Nation",
                     geography="England", 
                     metric="MMR1_coverage_coverageByYear")
    mmr1_coverage=api.get_all_pages()
    return mmr1_coverage # Return data read from the API

def access_api_mmr2(): # Access API data from UKHSA for MMR2 percentages
    """ Accesses the UKHSA API for MMR2. Returns data as a like-for-like replacement 
    for the "canned" data loaded from the JSON file. """
    api=APIwrapper(theme="immunisation", # Structure that contains the main parameters of the query  
                     sub_theme="childhood-vaccines",
                     topic="MMR2",
                     geography_type="Nation",
                     geography="England", 
                     metric="MMR2_coverage_coverageByYear")
    mmr2_coverage=api.get_all_pages()
    return mmr2_coverage # Return data read from the API

In [16]:
# Below function and button set up a refresh button to refresh the API data for measles cases, MMR1 and MMR2. 

def api_button_callback(button):
    """ Button callback - takes the button as its parameter.
    Accesses API, wrangles data, updates global variable df and adf used for plotting. """
    # Get data from the API
    global adf, cases_jsondata, df, mmr1_jsondata, mmr2_jsondata
    # After click, button will look like the following
    button.icon="spinner"
    button.description="Refreshing..."
    button.disabled=True # User cannot click again
    try:
        # Access the API for measles cases, MMR1 and MMR2, assign to JSON files
        cases_jsondata["measles_cases"]=access_api_measles() 
        mmr1_jsondata["mmr1_coverage"]=access_api_mmr1()
        mmr2_jsondata["mmr2_coverage"]=access_api_mmr2()
        # Wrangle the data and overwrite the dataframe for plotting
        adf=wrangle_data_measles_age(cases_jsondata["measles_cases"]) 
        df=wrangle_data_mmr(mmr1_jsondata["mmr1_coverage"], mmr2_jsondata["mmr2_coverage"])
        # Refresh the widgets of the measles cases age group graph   
        year_age.options=adf.index.year.unique()
        year_age.value=year_age.options[-1]
        # Refresh the widgets of the MMR graph
        year_select_mmr.options=df.index.year.unique()
        old_start,old_end=year_select_mmr.index
        if old_start < year_select_mmr.options[0] or old_end > year_select_mmr.options[-1]:
            year_select_mmr.index=(0,len(year_select_mmr.options)-1)
        # If data refreshed, button will look like the following
        button.icon="check"
        button.button_style="success"
        button.tooltip="Refresh successful"
        button.description="Updated"
        button.disabled=True
    # If data refresh fails, button will look like the following
    except Exception as e:
        button.description="Unavailable"
        button.button_style="danger"
        button.tooltip="Refresh failed"
        button.icon="times"
        return
# Initial refresh button will look like the following
apibutton=wdg.Button(
    description='Refresh Data',
    disabled=False,
    button_style='info',
    tooltip="Refresh UKHSA data for measles cases and MMR",
    icon='download'
)
apibutton.on_click(api_button_callback) # Name of function inside the brackets
display(apibutton)
# Run all cells before clicking on this button

Button(button_style='info', description='Refresh Data', icon='download', style=ButtonStyle(), tooltip='Refresh…

---------

## Graphs and Analysis

This section contains two graphs each with an interactive control, a figure description, and an analysis. Please read the Instructions subsection before continuing. 

### Instructions

The following two graphs contain data regarding measles cases per month by age group and MMR1 and MMR2 vaccination coverage percentages by year. Both graphs contain interactive controls that will allow the user to change the data view.  
- **Figure 1**: The interactive control in Figure 1 is a dropdown for year. The user should use the dropdown to toggle between years to view measles cases by age group for a particular year. 
- **Figure 2**: The interactive control in Figure 2 is a year range slider. The user should use the right and left buttons of the slider to view specific ranges of years for MMR1 and MMR2 vaccination coverage percentages. Once the user has moved the buttons to the desired year range, the user can also drag the slider from right and left to view a set range over different years. 

### Measles Cases per Month by Age Group

In [17]:
# Create widget for year selection
year_age=wdg.Select(
    options=adf.index.year.unique(), # Year options in dropdown are from the adf DataFrame and are unique 
    value=adf.index.year[-1], # Most recent year
    rows=1, # Rows of the selection box
    description='Year:',
    disabled=False
)
controls=wdg.HBox([year_age]) 

# Function for horizontal bar graph of measles cases by age group
def age_graph(graphyear): 
    yeardf=adf[adf.index.year==graphyear] # Callback function 
    monthly=yeardf.groupby(pd.Grouper(freq='1ME')).mean() # Average the rows by month end
    totals=monthly.sum(axis=1) # Sum over the rows
    monthly=monthly.div(totals, axis=0)*100 # Normalized to 100 
    monthly=monthly[::-1] # Older dates on top of the graph
    with plt.style.context('ggplot'): # Gray background formatting for graph and legend
        ax=monthly.plot(kind='barh', stacked=True, cmap='tab20b') # Horizontal stacked bar chart with color map tab20b
        ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), title='Ages') # Legend format
        ax.set_title(f"Measles Cases for {graphyear} by Age Group in England") 
        ax.set_xlabel('Cases')
        ax.set_yticklabels(monthly.index.strftime('%Y-%m-%d'))
        plt.show() # Graph won't update if this is missing 

output=wdg.interactive_output(age_graph, {'graphyear': year_age})
display(year_age, output)

Select(description='Year:', index=2, options=(2023, 2024, 2025), rows=1, value=2025)

Output()

**Figure 1. Comparison of average measles cases per month by age group in England.** This measles case metric is based on the number of weekly laboratory confirmed cases of measles as reported by UKHSA. This figure analyzes a time series slice of the query parameter 'age' from the overall cases metric. This figure groups the cases by month, takes their average, and normalizes them to 100 to fill the bar stack for an equal comparison across all age groups. The dates on the y-axis are sorted with the oldest date on top. A dropdown for year is shown at the top of the figure. The user is able to toggle between respective years to compare cases by age group across the different months and years that the data is collected for. By default, the dropdown will start at the most recent year. 

#### Analysis of Figure 1

When comparing measles cases across the current available years (2023-2025), age groups 01-04 and 05-10 make up the largest portion of average measles cases per month. When analyzing specific months, in September 2025 the 01-04 age group had an increase in average measles cases, as did the 15-24 age group in October of the same year. In October 2024, the 05-10 age group had an increase in average measles cases, whereas the 35+ age group saw a decrease in average measles cases for the same month. The data is limited for 2023. The only specified age group for September 2023 was the 15-24 age group. This is possibly due to gaps in data reporting.

### MMR Vaccination Coverage Percentages per Year

In [18]:
# Create widget for selection of year range as a slider 
year_select_mmr=wdg.SelectionRangeSlider( 
    options=df.index.year.unique(), # Year options in slider are from df DataFrame and are unique 
    index=(0,len(df.index.year.unique())-1), # Initial selection set to full range of years available
    description='Year:',
    disabled=False
)
controls=wdg.VBox([year_select_mmr])

# Function for line plot of MMR1 and MMR2
def timeseries_graph_mmr(gyear):
    start,end=gyear 
    years=pd.Series(data=df.index.year, index=df.index) # Create pandas Series from DataFrame index
    yeardf=df[years.between(start,end)] # Use pd method between to return True/False where year is between the start/end    
    with plt.style.context('ggplot'): # Gray background formatting for graph and legend
        yeardf.plot(linestyle='--', marker='o')
        plt.title(f"MMR Vaccination Coverage Percentages\n for {start}-{end} in England")
        plt.xlabel('Date')
        plt.ylabel('Vaccine Coverage Percentage')
        plt.legend(title='Vaccine', loc='upper left', bbox_to_anchor=(1.0,0.65)); # Legend format
        plt.show() # Graph won't update if this is missing  

graph=wdg.interactive_output(timeseries_graph_mmr, {'gyear': year_select_mmr})
display(controls, graph)

Output()

**Figure 2. Comparison of MMR1 and MMR2 vaccination coverage percentages per year in England.** The MMR1 vaccination metric represents the percentage of children who received the first dose of the MMR vaccine by 24 months and 5 years of age. The MMR2 vaccination metric represents the percentage of children who received the second dose of the MMR vaccine by 5 years of age. This figure compares MMR1 and MMR2 vaccination coverage percentages across several years of available data. A date range slider is displayed at the top of the figure. The user is able to slide the left and right buttons to compare specific ranges of years that the data is collected for. By default, the graph will display the full range of available years. 

#### Analysis of Figure 2

When comparing MMR vaccination coverage percentages together, in 2016 MMR1 and MMR2 both had high coverage percentages, where for MMR1 it was 94.8% and for MMR2 it was 88.23%. In 2010, MMR1 and MMR2 both had low coverage percentages, where for MMR1 it was 91.01% and for MMR2 it was 82.69%. Overall, the percentage of eligible children who receive the first dose of MMR is higher than the percentage of eligible children who receive the second dose. The highest MMR1 coverage percentage was 95% in 2017, whereas the highest MMR2 coverage percentage was 88.62% in 2015. The lowest MMR1 coverage percentage was 89.35% in 2023, whereas the lowest MMR2 coverage percentage was 82.69% in 2010.

### Combined Analysis of Figure 1 and Figure 2

The MMR vaccination metrics tracked in Figure 2 look at eligible children receiving the first and second doses of the MMR vaccine by the ages of 2 and 5. Specifically looking at the years the measles case data is collected for, in Figure 2, from 2023-2025 there is a general upward trend for MMR1 coverage percentages and a downward trend for MMR2 coverage percentages. The two age groups from Figure 1 that consistently made up the largest portion of average measles cases per month were ages 01-04 and 05-10. The downward trend of the MMR2 coverage percentages could signify that children are not receiving the second MMR dose by the age of 5 which is required to reach full protection against measles. Subsequently, these age groups would have higher measles susceptibility which would attribute to the higher average monthly measles case data seen in Figure 1. 

---------

U.K. Measles and MMR Vaccine Tracking Dashboard (C) Shyanne Herman, 2025. Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).